In [1]:
import pandas as pd
excel_files = ["batteryManagement.csv", "accelerometer.csv", "gyroscope.csv","multivibrators.csv","networkdevices.csv","motors.csv","microcontroller.csv"]
dfs = []
for file in excel_files:
    df = pd.read_csv(file)
    df['category']=file.split(".")[0]
    dfs.append(df)
merged_df = pd.concat(dfs, ignore_index=True)

merged_df

,product_name,manufacturer,cleaned_datasheet_text,category
0,ADBMS6948,Analog Devices,preliminary data measures up to battery cells ...,batteryManagement
1,ADBMS6830,Analog Devices,data multicell battery monitor typical applica...,batteryManagement
2,ADRF8800,Analog Devices,data low power ghz wireless system on chip low...,batteryManagement
3,ADBMS6816,Analog Devices,typical application circuit data multicell bat...,batteryManagement
4,LTC2959,Analog Devices,power battery gas gauge descriptionthe is an p...,batteryManagement
...,...,...,...,...
900,ADUC836,Analog Devices,dual adcs with embedded kb flash mcu featuresh...,microcontroller
901,ADUC814,Analog Devices,small adc with embedded flash analog ksps adc ...,microcontroller
902,ADUC816,Analog Devices,microconverter adcs with embedded flash mcu ad...,microcontroller
903,ADUC824,Analog Devices,microconverter adcs with embedded flash mcu fe...,microcontroller


In [13]:
merged_df.head(400)

,product_name,manufacturer,cleaned_datasheet_text,category
0,ADBMS6948,Analog Devices,preliminary data measures up to battery cells ...,batteryManagement
1,ADBMS6830,Analog Devices,data multicell battery monitor typical applica...,batteryManagement
2,ADRF8800,Analog Devices,data low power ghz wireless system on chip low...,batteryManagement
3,ADBMS6816,Analog Devices,typical application circuit data multicell bat...,batteryManagement
4,LTC2959,Analog Devices,power battery gas gauge descriptionthe is an p...,batteryManagement
...,...,...,...,...
395,ADXRS620BBGZ,Analog Devices Inc.,qualified for automotive applications complete...,gyroscope
396,FXAS21002CQR1,NXP USA Inc.,freescale semiconductor sheet technical data d...,gyroscope
397,ADXRS649BBGZ,Analog Devices Inc.,data sheet high vibration rejection over wide ...,gyroscope
398,ADIS16265BCCZ,Analog Devices Inc.,data sheet programmable digital gyroscope feat...,gyroscope


In [14]:
unique_values = merged_df['category'].unique()

In [15]:
len(unique_values)

7

In [16]:
merged_df.isnull().sum()

product_name              0
manufacturer              0
cleaned_datasheet_text    1
category                  0
dtype: int64

In [2]:
merged_df.dropna(inplace=True)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

X = merged_df['cleaned_datasheet_text']
y = merged_df['category']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,stratify=y, random_state=42)

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)



In [12]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)
y_pred = nb_classifier.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
report = classification_report(y_test, y_pred)
print(report)

Accuracy: 0.7955801104972375
                   precision    recall  f1-score   support

    accelerometer       0.59      0.98      0.74        45
batteryManagement       1.00      0.94      0.97        33
        gyroscope       0.00      0.00      0.00         9
  microcontroller       0.00      0.00      0.00         8
           motors       1.00      0.45      0.62        20
   multivibrators       0.00      0.00      0.00         6
   networkdevices       0.90      1.00      0.94        60

         accuracy                           0.80       181
        macro avg       0.50      0.48      0.47       181
     weighted avg       0.74      0.80      0.74       181



C:\Users\soumi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\soumi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\soumi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)
y_pred = rf_classifier.predict(X_test_tfidf)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
report = classification_report(y_test, y_pred)
print(report)

Random Forest Accuracy: 0.9447513812154696
                   precision    recall  f1-score   support

    accelerometer       0.94      0.98      0.96        45
batteryManagement       1.00      0.97      0.98        33
        gyroscope       1.00      0.78      0.88         9
  microcontroller       1.00      1.00      1.00         8
           motors       1.00      0.70      0.82        20
   multivibrators       1.00      1.00      1.00         6
   networkdevices       0.90      1.00      0.94        60

         accuracy                           0.94       181
        macro avg       0.98      0.92      0.94       181
     weighted avg       0.95      0.94      0.94       181



In [14]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_tfidf, y_train)
y_pred = svm_classifier.predict(X_test_tfidf)
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
report = classification_report(y_test, y_pred)
print(report)

SVM Accuracy: 0.9613259668508287
                   precision    recall  f1-score   support

    accelerometer       0.96      0.96      0.96        45
batteryManagement       1.00      0.97      0.98        33
        gyroscope       1.00      0.89      0.94         9
  microcontroller       1.00      1.00      1.00         8
           motors       1.00      0.90      0.95        20
   multivibrators       1.00      1.00      1.00         6
   networkdevices       0.92      0.98      0.95        60

         accuracy                           0.96       181
        macro avg       0.98      0.96      0.97       181
     weighted avg       0.96      0.96      0.96       181

